In [ ]:
# You need to run this cell twice or else the following cells won't run.
import keras
from keras.layers import Dense, Dropout, Activation
import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from IPython.display import FileLink, FileLinks
import matplotlib.pyplot as plt
import pandas as pd
import sys

In [ ]:
# 1 image has 6x6 pixels = 36 pixels 
pixels = ["pixel_{0}".format(i) for i in range(36)]
def to_image(df):
    return  np.expand_dims(np.expand_dims(df[pixels], axis=-1).reshape(-1,6,6), axis=-1)

# You need to email hichemb@princeton.edu to request access to this directory, or else this line will not work
df = pd.read_hdf("/eos/user/h/hboucham/SWAN_projects/MergedHits/output_final.h5", key="df", mode='r') # do not write to this file !

# If you want to run "file.h5" locally use the following lines:
# store_train = pd.HDFStore("file.h5")
# df = store_train.select("df" , stop = -1) # df here is like a tree1 in Root

#Last Cuts and selections:
#cut on DeltaR and nUniqueSimTracksInSharedHit
df = df[(df["GenDeltaR"]<0.1) & (df["nUniqueSimTracksInSharedHit"]>-1)]
df_old = df

In [ ]:
#Removing events that have single pixel since the neural network cannot tell whether it's merged or not, there is simply not enough information.
pixelColumns = ["pixel_%i" % x for x in range(36)]
pixels_df = df[pixelColumns].values
# This printout is a sanity check, there 36 pixels so we expect pixels_df to have 36 columns
#print(pixels_df[0].shape)
pixel_number = pixels_df.astype(bool).sum(axis=1)
df.insert(0, "Pixel_number", pixel_number)
# the df.info() printouts allows us to see how many events were removed.
#print df.info()
df = df[df["Pixel_number"]>1]
#print df.info()

In [ ]:
# frac=0.5 sets half training and half testing
df_train=df.sample(frac=0.5)
df_test=df.drop(df_train.index)
images_train = to_image(df_train)
images_test = to_image(df_test)

In [ ]:
# Sanity check: printout (events, variables) Variables include 36 pixels, and a few others you can find using df.info() command
print "Test Data Shape: ",df_test.shape
print "Train Data Shape: ",df_train.shape
#print df.info()

In [ ]:
# initializing a log file for your training, which will be used for the loss vs epoch plot
csv_logger = tf.keras.callbacks.CSVLogger('training.txt', separator=",", append=False)

In [ ]:
# This is the CNN
# Adding layers to Neural Network: (1) is convolutional,(1.5) 2D layer ,(2) flatten output then feed it to (3) which is a regular neural network.
# (4) drops nodes in NN to avoid overfitting, finallly (5) outputs 2 values (prob(notmergedhit), prob(merged hit)), must add up to 1.

# Define the network
model = keras.models.Sequential()

#layer (1)
model.add(keras.layers.Conv2D(32, kernel_size=(4,4), padding='same', activation='relu'))

#layer (1.5) : you can play with these layers individually or together until you find the best combination 
#model.add(keras.layers.Conv2D(16, kernel_size=(4,4), padding='same', activation='relu'))
#model.add(keras.layers.Conv2D(16, kernel_size=(2,2), padding='same', activation='relu'))
#model.add(keras.layers.Conv2D(16, kernel_size=(1,1), padding='same', activation='relu'))

# layer (X): max pooling 2D, has always been commented out, but you can uncomment it and see what it does
#model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same', data_format="channels_last"))

# layer (2)
model.add(keras.layers.Flatten(input_shape=(6,6,1),data_format = "channels_last"))

# layer (3)
model.add(keras.layers.Dense(50, activation='relu'))

# layer 4, dropout 10%
model.add(Dropout(0.1))
model.add(keras.layers.Dense(2, activation='softmax'))
                                                    
# Layer (5), train the network
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics = ["accuracy"]) 
# validation fraction is fraction of training sample used for testing (here 10%), epochs: number of times you run the CNN.
epochs_number = 100 # change the number of epochs used in the training HERE
model.fit(images_train, keras.utils.to_categorical(df_train["nUniqueSimTracksInSharedHit"]>1), callbacks=[csv_logger], epochs=epochs_number, validation_split=0.1) 

# Prints out a summary of the training at the end
model.summary()

In [ ]:
# Loss vs Epoch Plot script

loss = []
with open("training.txt" )  as file:
     for line in file:
           # skipping the first line of the file since it contains column name ("epoch", "loss"..etc)
            n = line.find("epoch")
            if n != -1:
                continue
            n = line.find(",",6, len(line)-1)
            if n != -1:
                x = line[n+1:n+ 7]
                x = float(x)
                loss.append(x)
                continue
#print(loss)

epoch = []
for i in range(0,epochs_number): # 100 epoch number
    epoch.append(float(i+1))   
#print(epoch)
   
p = np.poly1d(np.polyfit(epoch, loss, 10)) # using 10 degrees of freedom for our fit
t = np.linspace(1,epochs_number, epochs_number*20) 
plt.plot(epoch, loss, 'o', t, p(t),  '-')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss vs Epoch Plot")
plt.show()

In [ ]:
# Evaluate performance on independent sample, using CNN on Test sample
ret = model.predict(images_test)
np.save("result.pynb",ret[:,1]) 
#print(ret)

In [ ]:
# evaluating ROC curve inputs
fpr_keras, tpr_keras, thresholds_keras = roc_curve(keras.utils.to_categorical(df_test["nUniqueSimTracksInSharedHit"]>1)[:,1], ret[:,1])
#print fpr_keras,tpr_keras
auc_keras = auc(fpr_keras, tpr_keras)
#print auc_keras
#print np.isnan(fpr_keras).all()
#print len(fpr_keras),len(tpr_keras)
np.save("fpr_keras.npy",fpr_keras)
np.save("tpr_keras.npy",tpr_keras)

In [ ]:
#Plotting ROC curve
fpr_keras = np.load("fpr_keras.npy")
tpr_keras = np.load("tpr_keras.npy")
auc = np.trapz(tpr_keras,fpr_keras)
#print auc

plt.figure(1)
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC Curve (area = {:.3f})'.format(auc))
plt.savefig("ROC.png")
plt.savefig("ROC.pdf")
plt.show() 

In [ ]:
# Sanity check: no overlap events between train and test 
pd.merge(df_train, df_test, on=[x for x in df_train.columns], how='inner')

In [ ]:
# Visualizing our training data by number of hits
print "shared hits with at least 2 sim tracks:"
print float(sum(df_train["nUniqueSimTracksInSharedHit"]>1))/len(df_train["nUniqueSimTracksInSharedHit"])
plt.hist(df_train["nUniqueSimTracksInSharedHit"],histtype="step",bins=6,range=(-0.5,5.5))
plt.xlabel('Hits')
plt.ylabel('Events')
plt.title("Distribution of Hits in Training Data")

In [ ]:
# Visualizing our training data by merged (2 hits or more) vs not merged (1 hit) 
print "shared hits with at least 2 sim tracks:"
print float(sum(df_train["nUniqueSimTracksInSharedHit"]>1))/len(df_train["nUniqueSimTracksInSharedHit"])
plt.hist(df_train["nUniqueSimTracksInSharedHit"]>1,histtype="step",bins=2,range=(-0.5,1.5))
plt.title("Distribution of Hits in Training Data")
plt.xticks([0,1],("Not Merged","Merged"))
plt.ylabel('Events')
plt.savefig("merged_dist.png")
plt.savefig("merged_dist.pdf")

In [ ]:
# Separating signal and background for train and test data the preparing histograms for discriminant plot
# Testing data
signal = ret[df_test["nUniqueSimTracksInSharedHit"]>1]
background = ret[df_test["nUniqueSimTracksInSharedHit"]<2]
signal_plt = signal[:,1]
background_plt = background[:,1]

#Training data
ret_train = model.predict(images_train)
signal_train = ret_train[df_train["nUniqueSimTracksInSharedHit"]>1]
background_train = ret_train[df_train["nUniqueSimTracksInSharedHit"]<2]
signal_train_plt = signal_train[:,1]
background_train_plt = background_train[:,1]
Y_back_hist = np.histogram(background_train_plt, bins = 30, range = (0,1))[0]
X_back_hist = np.histogram(background_train_plt, bins = 30, range = (0,1))[1]
Y_sig_hist = np.histogram(signal_train_plt, bins = 30, range = (0,1))[0]
X_sig_hist = np.histogram(signal_train_plt, bins = 30, range = (0,1))[1]

In [ ]:
# Plotting Signal and Background Discriminants 
plt.hist(signal_plt, alpha = 0.5, color = 'b', label = 'Signal (test)', range = (0,1), bins = 30)
plt.hist(background_plt, color = 'r', alpha = 0.5, label = 'Background (test)', range = (0,1), bins = 30)
plt.scatter(X_back_hist[0:30] + 0.0166 , Y_back_hist, label='Background (train)', color ='r')
plt.scatter(X_sig_hist[0:30] + 0.0166, Y_sig_hist, label='Signal (train)', color='b')
plt.legend(loc='best')
plt.xlabel('Discriminant')
plt.ylabel('Events')
plt.title('CNN Signal and Background Discriminants')

In [ ]:
# Adding Discriminant branch to Testing data only, could easily do it for Training data but we don't need to (for our purpose below)
n_events_df_test = len(df_test.index) # number of events in df_test
disc = []
discriminants_test = model.predict(images_test) # returns (prob(notmergedhit), prob(merged hit)), the second number is our discriminant
for i in range (0,n_events_df_test):
    disc.append(discriminants_test[i][1]) 
df_test.insert(0, "Discriminants", disc) #inserting new column in our dataframe at position 0

In [ ]:
# Sanity check: let's see if Discriminants is non zero and it's at the 0th position
#print df_test.head()
#print df_test.info()

In [ ]:
# Pixel Picture Script
def cluster_map(data_f):
    # sorting by discriminant ascending
    data_f = data_f.sort_values("Discriminants", ascending = True)[0:len(data_f.index)-1]

    shareds =  data_f
    pixelColumns = ["pixel_%i" % x for x in range(36)]
    pixels = shareds[pixelColumns].values

    for row,hit,  in enumerate(pixels):
        x_pos = []
        y_pos = []
        charge = []
        for index,pixel in enumerate(hit):
            if pixel!=0:
                x_pos.append(index%6)
                y_pos.append(np.floor(index/6))  
                charge.append(pixel)
        dis = np.around(data_f.iloc[row,0], decimals = 5)
        text = "     Event " + str(row +1) + " with discriminant " + str(dis)
        print text
        
        # Plotting Colorbar    
        fig=plt.figure()
        ax=plt.axes()
        cax = plt.hist2d(x_pos,y_pos,weights=charge,bins=(6,6),range=((0,6),(0,6)))
        cb=fig.colorbar(cax[3])
        cb.set_ticks([0,max(charge)])
        cb.set_label("normalized adc",rotation=-90)

        # Title, uses truth value
        hits_column = data_f.columns.get_loc("nUniqueSimTracksInSharedHit")
        if data_f.iloc[row,hits_column] == 1 : # 1 hit
            plt.title('         Not Merged Cluster Charge Map (discriminant = {:.3f})'.format(dis))
        elif data_f.iloc[row,hits_column] > 1 : # 2 or more hits
            plt.title('         Merged Cluster Charge Map (discriminant = {:.3f})'.format(dis))
        else : # 0 hits
            plt.title('         Null Cluster Charge Map (discriminant = {:.3f})'.format(dis))
    
        plt.xlabel("x")
        plt.ylabel("y")
        plt.show()

In [ ]:
# Pixel Picture Script for [0, 0.1] discriminant testing events
data = df_test [(df_test["Discriminants"] < 0.1 )] 
cluster_map(data)

In [ ]:
# Distribution of 0 - 0.1 discriminant events
data["Discriminants"].plot(kind='hist', title = "Discriminant in the Range [0, 0.1]", bins = 100, figsize=(12,6))

In [ ]:
# Pixel Picture Script for [0.3, 0.5] discriminant testing events
data = df_test [(df_test["Discriminants"] > 0.4 ) & (df_test["Discriminants"] < 0.6 ) ] 
cluster_map(data)

In [ ]:
# Distribution of 0.4 - 0.6 discriminant events
data["Discriminants"].plot(kind='hist', title = "Discriminant in the Range [0.3, 0.5]", bins = 100, figsize=(12,6))

In [ ]:
# Pixel Picture Script for [0.9, 1] discriminant testing events
data = df_test [df_test["Discriminants"] > 0.9 ] 
cluster_map(data)

In [ ]:
# Distribution of 0.9 - 1 discriminant events
data["Discriminants"].plot(kind='hist' , title = "Discriminant in the Range [0.9, 1]", bins = 100, figsize=(12,6))

In [ ]:
# Width and height script

# including all testing events
data = df_test 
# sorting by discriminant ascending
data = data.sort_values("Discriminants", ascending = True)[0:len(data.index)-1]

shareds =  data
pixelColumns = ["pixel_%i" % x for x in range(36)]
pixels = shareds[pixelColumns].values
width = [] # dx
height = [] # dy

for row,hit,  in enumerate(pixels):
    x_pos = []
    y_pos = []
    charge = []
    arra = np.zeros((6,6))
    for index,pixel in enumerate(hit):   
        if pixel!=0:
            x_pos.append(index%6)
            y_pos.append(np.floor(index/6))  
            charge.append(pixel)
            arra [5 - int(np.floor(index/6))][int(index%6)]= pixel
    
    #Evaluating width and height of every event
    charge_in_x = np.sum(arra,axis=0)
    charge_in_y = np.sum(arra,axis=1)
    charge_x_values = np.where(charge_in_x>0)[0]
    charge_y_values = np.where(charge_in_y>0)[0]
    wid = charge_x_values[-1] - charge_x_values[0] + 1
    hei = charge_y_values[-1] - charge_y_values[0] + 1
    width.append(wid)
    height.append(hei)

 
 # Uncomment this section to display each event information and pixel pictures
     # Event info
#    text = "Event " + str(row +1) + " with discriminant " + str(np.around(data.iloc[row,0], decimals = 5)) + ", width "+ str(wid)+ " and height " +str(hei)
#    print text
#    # Plotting Colorbar  
#    fig=plt.figure()
#    ax=plt.axes()
#    cax = plt.hist2d(x_pos,y_pos,weights=charge,bins=(6,6),range=((0,6),(0,6)))
#    cb=fig.colorbar(cax[3])
#    cb.set_ticks([0,max(charge)])
#    cb.set_label("normalized adc",rotation=-90)
#
#    # Title, uses truth value
#    hits_column = df_test.columns.get_loc("nUniqueSimTracksInSharedHit")
#    if data.iloc[row,hits_column] == 1 : # 1 hit
#        plt.title("Not Merged Cluster Charge Map")
#    elif data.iloc[row,hits_column] > 1 : # 2 or more hits
#        plt.title("Merged Cluster Charge Map")
#    else : # 0 hits
#        plt.title("Null Cluster Charge Map")
#    
#    plt.xlabel("x")
#    plt.ylabel("y")
#    plt.show() 



In [ ]:
# Adding width and height branch and checking they are there
data.insert(1, "Height", height)
data.insert(1, "Width", width)
#print data.info()

In [ ]:
# Separating signal and background for testing data
signal = data[(data["nUniqueSimTracksInSharedHit"]>1)]
background = data[(data["nUniqueSimTracksInSharedHit"]<2)]

# Plotting CNN Signal and Background Width
signal_plt_width = signal["Width"]
background_plt_width = background["Width"]
plt.hist(signal_plt_width, alpha = 0.5, color = 'b', label = 'Signal (test)', range = (1,6), bins = 6)
plt.hist(background_plt_width, color = 'r', alpha = 0.5, label = 'Background (test)', range = (1,6), bins = 6)
plt.legend(loc='best')
plt.title('CNN Signal and Background Width')
plt.xlabel('Width')
plt.ylabel('Events')

In [ ]:
# Plotting CNN Signal and Background Height
signal_plt_height = signal["Height"]
background_plt_height = background["Height"]
plt.hist(signal_plt_height, alpha = 0.5, color = 'b', label = 'Signal (test)', range = (1,6), bins = 6)
plt.hist(background_plt_height, color = 'r', alpha = 0.5, label = 'Background (test)', range = (1,6), bins = 6)
plt.legend(loc='best')
plt.title('CNN Signal and Background Height')
plt.xlabel('Height')
plt.ylabel('Events')